#  漲停後 one-shot 天內問【今天續漲嗎？】改用貝葉斯看看

- 照老師的方法，把資料整理成 150天 一長條一筆，如果往前沒有 150 天就不用。
- 掃出所有漲停的日子，以及後 one-shot 天，如果又有漲停就繼續 one-shot。這樣縮小 dataset 範圍。
- 以此 dataset 問 AI 明天是否【續漲】且【不破前低】？平衡正反雙方，取得 n 值。

問題很奇怪, DNN 的 prediction 全是 0 ! 想要改用別的方法做做看

In [1]:
%%js
alert("要不要把 model/DNN directory 先殺掉？")
alert("注意 batch_size 合不合理？一千也照跑，結果都是 0.")

<IPython.core.display.Javascript object>

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import peforth as f

In [3]:
%%f
\ Now we redefine the 'unknown' command that does nothing at default  
: unknown ( token -- thing Y|N) // Try to find the unknown token in __main__
  py> getattr(sys.modules['__main__'],pop(),"Ûnknôwn") 
  py> str(tos())=="Ûnknôwn" if drop false else true then ;
  /// here after, when FORTH come accross an unknown token, instead of alerting 
  /// it try to find the token in python __main__ module name space.
: path-to-find-modules ( <path> -- ) // Add path to sys.path so "import module-name" can find the module
    CR word trim ( "path" ) py: sys.path.append(pop()) ;
code \ print(nexttoken('\n')) end-code // Redefine \ command to print the comment line 


reDef unknown
reDef \


In [4]:
data = pd.read_csv('000777.csv',encoding='gbk',parse_dates=[0],index_col=0)

# data frame 中帶有 None 的 rows 都剔除掉。 本來有 4752 rows 現在只剩 4639 
# 用法: df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
data = data.replace(to_replace='None', value=np.nan).dropna()  

# pdb.set_trace() # data 本來是新的在上面
data.sort_index(0,ascending=True,inplace=True) # 排序 sort 過後變成最舊的在上面

# 上市第一天的資料不要，因為【漲跌幅】無效。
data = data.query('index > "1997-07-10"')

In [5]:
# raw data has 20 years long, but here only analysis the last 150 days of a chosen day
dayfeature = 150
oneshot = 5  # 漲停後慣性天數，使正反平衡
dayforesee = oneshot  # 問未來幾天如何如何，最後就要扣掉幾天，避免 out of bounds. 0 代表當天。
featurenum = 5*dayfeature  # 取五個有關係的欄位，ochlq 等五個特徵值。
cross_validation_rate = 0.2
xlength = data.shape[0]-dayfeature-dayforesee

In [6]:
%f ( 總天數 ) data :> shape[0] tib. ( 4638 )
%f ( leading days that are not used ) dayfeature tib.   
%f ( latest days that are not used ) dayforesee tib.
%f ( 有效天數 ) data :> shape[0] dayfeature - dayforesee - tib.

( 總天數 ) data :> shape[0] tib. \ ==> 4638 (<class 'int'>)
( leading days that are not used ) dayfeature tib. \ ==> 150 (<class 'int'>)
( latest days that are not used ) dayforesee tib. \ ==> 5 (<class 'int'>)
( 有效天數 ) data :> shape[0] dayfeature - dayforesee - tib. \ ==> 4483 (<class 'int'>)


In [7]:
x=np.zeros((xlength,featurenum+2))  # +2 是當天開盤跟當天日期
y=np.zeros((xlength));

In [8]:
# 重疊 shift dayfeature (150) 天的五個「行情特徵值」加上當天的開盤 共 751 個參數當作 feature X 
for i in range(0,xlength):
    x[i,0:featurenum]=np.array(data[i:i+dayfeature] \
          [[u'收盘价',u'最高价',u'最低价',u'开盘价',u'成交量']]).reshape((1,featurenum))
    x[i,featurenum]=data.iloc[i+dayfeature][u'开盘价']
    x[i,featurenum+1]=data.index[i+dayfeature].value  # backward pd.to_datetime(value).date()
    

In [9]:
# 預測明天是否會「續漲」且「不破前低」? ochl 
for i in range(0,xlength):
    y[i]=0              
    i0h = float(data.iloc[i][u'最高价']) # i 這天的
    i0l = float(data.iloc[i][u'最低价']) # i 這天的
    i1h = float(data.iloc[i+1][u'最高价']) # i 明天的
    i1l = float(data.iloc[i+1][u'最低价']) # i 明天的
    if i1h > i0h and i1l > i0l :
            y[i]=1
y = np.int32(y)

In [10]:
%f (  ) x :> shape tib. ( ?, 751 )
%f (  ) x :> shape[0] tib. ( ? )
%f (  ) y :> shape tib. ( ?, )    
%f (  ) y :> shape[0] tib. ( ? )    
%f (  ) y py> sum(pop()) tib. ( 1629 )

(  ) x :> shape tib. \ ==> (4483, 752) (<class 'tuple'>)
(  ) x :> shape[0] tib. \ ==> 4483 (<class 'int'>)
(  ) y :> shape tib. \ ==> (4483,) (<class 'tuple'>)
(  ) y :> shape[0] tib. \ ==> 4483 (<class 'int'>)
(  ) y py> sum(pop()) tib. \ ==> 1629 (<class 'numpy.int32'>)


In [11]:
# 直接看【涨跌幅】> 9.8 就算漲停板，從【漲停板】的日子往後多取 oneshot 天。
# 符合條件的日子組成 dataset 其中含有 oneshot 天慣性，使正反兩方平衡。
# 這個簡單的 loop 是挑出要的日子且消除重複的方法，我考慮過用 set() 也是取 unique 的好方法。

x_mask = np.zeros((xlength,)) # 全部放 0 以下只針對目標填 1 
x_mask = np.int32(x_mask)
for i in range(xlength):
    if float(data.iloc[i+dayfeature-1][u'涨跌幅'])>=9.8:
        for j in range(oneshot):
            x_mask[i+j-1] = 1

In [12]:
%f x_mask tib.
%f x_mask :> shape tib.
%f x_mask py> sum(pop()) tib.

x_mask tib. \ ==> [0 0 0 ..., 0 0 0] (<class 'numpy.ndarray'>)
x_mask :> shape tib. \ ==> (4483,) (<class 'tuple'>)
x_mask py> sum(pop()) tib. \ ==> 350 (<class 'numpy.int32'>)


In [13]:
x_original = []
y_original = []
for i in range(xlength):
    if x_mask[i]:
        x_original.append(x[i])
        y_original.append(y[i])
x,x_original = x_original,x
y,y_original = y_original,y

In [14]:
%f x_original np :> array(pop()).shape tib.
%f y_original np :> array(pop()).shape tib.
%f x np :> array(pop()).shape tib.
%f y np :> array(pop()).shape tib.
%f ( 差不多是 dataset 總數的一半嗎？ ) y py> sum(pop()) tib.

x_original np :> array(pop()).shape tib. \ ==> (4483, 752) (<class 'tuple'>)
y_original np :> array(pop()).shape tib. \ ==> (4483,) (<class 'tuple'>)
x np :> array(pop()).shape tib. \ ==> (350, 752) (<class 'tuple'>)
y np :> array(pop()).shape tib. \ ==> (350,) (<class 'tuple'>)
( 差不多是 dataset 總數的一半嗎？ ) y py> sum(pop()) tib. \ ==> 156 (<class 'numpy.int32'>)


In [15]:
from sklearn import cross_validation
x_train, x_test, y_train, y_test = cross_validation.train_test_split(x, y, test_size = 0.2)

c:\users\hcche\appdata\local\programs\python\python36\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [16]:
import sklearn
sklearn.__version__

'0.19.1'

In [17]:
%f x_train np :> array(pop()).shape tib.
%f y_train np :> array(pop()).shape tib.
%f x_test np :> array(pop()).shape tib.
%f y_test np :> array(pop()).shape tib.
%f ( numpy 才能裁切 matrix ) x_train np :> array(pop())[:,:751].shape tib.
%f ( numpy 才能裁切 matrix ) x_test np :> array(pop())[:,:751].shape tib.


x_train np :> array(pop()).shape tib. \ ==> (280, 752) (<class 'tuple'>)
y_train np :> array(pop()).shape tib. \ ==> (280,) (<class 'tuple'>)
x_test np :> array(pop()).shape tib. \ ==> (70, 752) (<class 'tuple'>)
y_test np :> array(pop()).shape tib. \ ==> (70,) (<class 'tuple'>)
( numpy 才能裁切 matrix ) x_train np :> array(pop())[:,:751].shape tib. \ ==> (280, 751) (<class 'tuple'>)
( numpy 才能裁切 matrix ) x_test np :> array(pop())[:,:751].shape tib. \ ==> (70, 751) (<class 'tuple'>)


In [18]:
# 應該只有一個 feature 是個 shape = (751,) 的 array. 這怎麼表達?
my_feature_columns = [tf.feature_column.numeric_column("x", shape=[751,])]  
my_feature_columns

[_NumericColumn(key='x', shape=(751,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [19]:
# Build 2 hidden layer DNN with 10, 10 units respectively.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 10 nodes each.
    hidden_units=[64,32],
    # The model must choose between 3 classes.
    n_classes=2,
    dropout=0.2,
    model_dir="model/DNN"
    )

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'model/DNN', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000022621116BE0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [20]:
# Define the training inputs 從 mnist 抄過來改的
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": np.array(x_train)[:,:751]},
    y=np.array(y_train),
    num_epochs=None,
    batch_size=64,
    shuffle=True
    )

In [21]:
%pdb off
classifier.train(input_fn=train_input_fn, steps=30000);

Automatic pdb calling has been turned OFF
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into model/DNN\model.ckpt.
INFO:tensorflow:loss = 2.50763e+08, step = 1
INFO:tensorflow:global_step/sec: 334.799
INFO:tensorflow:loss = 141593.0, step = 101 (0.299 sec)
INFO:tensorflow:global_step/sec: 418.432
INFO:tensorflow:loss = 20543.8, step = 201 (0.243 sec)
INFO:tensorflow:global_step/sec: 474.202
INFO:tensorflow:loss = 2006.95, step = 301 (0.210 sec)
INFO:tensorflow:global_step/sec: 474.208
INFO:tensorflow:loss = 1642.81, step = 401 (0.216 sec)
INFO:tensorflow:global_step/sec: 454.806
INFO:tensorflow:loss = 2754.9, step = 501 (0.216 sec)
INFO:tensorflow:global_step/sec: 448.688
INFO:tensorflow:loss = 2314.37, step = 601 (0.222 sec)
INFO:tensorflow:global_step/sec:

INFO:tensorflow:global_step/sec: 427.593
INFO:tensorflow:loss = 99.6241, step = 8201 (0.235 sec)
INFO:tensorflow:global_step/sec: 458.981
INFO:tensorflow:loss = 43.2217, step = 8301 (0.217 sec)
INFO:tensorflow:global_step/sec: 413.457
INFO:tensorflow:loss = 43.9058, step = 8401 (0.244 sec)
INFO:tensorflow:global_step/sec: 365.175
INFO:tensorflow:loss = 56.7049, step = 8501 (0.274 sec)
INFO:tensorflow:global_step/sec: 436.93
INFO:tensorflow:loss = 43.6482, step = 8601 (0.228 sec)
INFO:tensorflow:global_step/sec: 465.384
INFO:tensorflow:loss = 44.0812, step = 8701 (0.216 sec)
INFO:tensorflow:global_step/sec: 431.28
INFO:tensorflow:loss = 43.9111, step = 8801 (0.229 sec)
INFO:tensorflow:global_step/sec: 444.699
INFO:tensorflow:loss = 42.6944, step = 8901 (0.226 sec)
INFO:tensorflow:global_step/sec: 433.149
INFO:tensorflow:loss = 43.5813, step = 9001 (0.232 sec)
INFO:tensorflow:global_step/sec: 450.709
INFO:tensorflow:loss = 131.365, step = 9101 (0.221 sec)
INFO:tensorflow:global_step/sec:

INFO:tensorflow:global_step/sec: 406.65
INFO:tensorflow:loss = 13588.2, step = 16601 (0.232 sec)
INFO:tensorflow:global_step/sec: 491.23
INFO:tensorflow:loss = 43.2174, step = 16701 (0.210 sec)
INFO:tensorflow:global_step/sec: 434.042
INFO:tensorflow:loss = 6599.68, step = 16801 (0.222 sec)
INFO:tensorflow:global_step/sec: 491.785
INFO:tensorflow:loss = 294.223, step = 16901 (0.220 sec)
INFO:tensorflow:global_step/sec: 431.183
INFO:tensorflow:loss = 43.7396, step = 17001 (0.212 sec)
INFO:tensorflow:global_step/sec: 378.262
INFO:tensorflow:loss = 43.4706, step = 17101 (0.268 sec)
INFO:tensorflow:global_step/sec: 450.711
INFO:tensorflow:loss = 43.9169, step = 17201 (0.221 sec)
INFO:tensorflow:global_step/sec: 452.749
INFO:tensorflow:loss = 43.0679, step = 17301 (0.221 sec)
INFO:tensorflow:global_step/sec: 408.891
INFO:tensorflow:loss = 43.0593, step = 17401 (0.245 sec)
INFO:tensorflow:global_step/sec: 460.499
INFO:tensorflow:loss = 43.9119, step = 17501 (0.213 sec)
INFO:tensorflow:global

INFO:tensorflow:global_step/sec: 401.838
INFO:tensorflow:loss = 44.0829, step = 25001 (0.248 sec)
INFO:tensorflow:global_step/sec: 469.604
INFO:tensorflow:loss = 43.0611, step = 25101 (0.210 sec)
INFO:tensorflow:global_step/sec: 424.114
INFO:tensorflow:loss = 43.5748, step = 25201 (0.236 sec)
INFO:tensorflow:global_step/sec: 403.07
INFO:tensorflow:loss = 44.4865, step = 25301 (0.257 sec)
INFO:tensorflow:global_step/sec: 462.217
INFO:tensorflow:loss = 44.5886, step = 25401 (0.207 sec)
INFO:tensorflow:global_step/sec: 423.463
INFO:tensorflow:loss = 72.6502, step = 25501 (0.252 sec)
INFO:tensorflow:global_step/sec: 432.073
INFO:tensorflow:loss = 43.912, step = 25601 (0.216 sec)
INFO:tensorflow:global_step/sec: 411.762
INFO:tensorflow:loss = 44.0823, step = 25701 (0.248 sec)
INFO:tensorflow:global_step/sec: 442.496
INFO:tensorflow:loss = 43.5707, step = 25801 (0.221 sec)
INFO:tensorflow:global_step/sec: 411.192
INFO:tensorflow:loss = 264.303, step = 25901 (0.246 sec)
INFO:tensorflow:global

In [22]:
# Define the test inputs  從 mnist 抄過來改的
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": np.array(x_test)[:,:751]},
    y=np.array(y_test),
    num_epochs=1,
    shuffle=False
    )

In [23]:
# Evaluate accuracy
evaluate = classifier.evaluate(input_fn=test_input_fn)
accuracy_score = evaluate["accuracy"]
print("\nTest Accuracy: {0:f}%\n".format(accuracy_score*100))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-06-09:52:13
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model/DNN\model.ckpt-30000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-06-06-09:52:14
INFO:tensorflow:Saving dict for global step 30000: accuracy = 0.6, accuracy_baseline = 0.6, auc = 0.5, auc_precision_recall = 0.7, average_loss = 0.67971, global_step = 30000, label/mean = 0.4, loss = 47.5797, precision = 0.0, prediction/mean = 0.457405, recall = 0.0

Test Accuracy: 60.000002%



In [24]:
evaluate

{'accuracy': 0.60000002,
 'accuracy_baseline': 0.60000002,
 'auc': 0.5,
 'auc_precision_recall': 0.69999999,
 'average_loss': 0.67970961,
 'global_step': 30000,
 'label/mean': 0.40000001,
 'loss': 47.579674,
 'precision': 0.0,
 'prediction/mean': 0.45740461,
 'recall': 0.0}

## How about a 'recall' ?

In [25]:
# Define the test inputs  從 mnist 抄過來改的
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": np.array(x_train)[:,:751]},
    y=np.array(y_train),
    num_epochs=1,
    shuffle=False
    )

In [26]:
# Evaluate accuracy
accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]
print("\nTest Accuracy: {0:f}%\n".format(accuracy_score*100))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-06-09:52:17
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model/DNN\model.ckpt-30000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-06-06-09:52:18
INFO:tensorflow:Saving dict for global step 30000: accuracy = 0.542857, accuracy_baseline = 0.542857, auc = 0.5, auc_precision_recall = 0.728571, average_loss = 0.689469, global_step = 30000, label/mean = 0.457143, loss = 64.3505, precision = 0.0, prediction/mean = 0.457405, recall = 0.0

Test Accuracy: 54.285717%



### 用全部資料來比對

In [27]:
# Make input function for prediction:
#   shuffle=False -> do not randomize input data
pred_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": np.array(x)[:,:751]},
    y=np.array(y),
    num_epochs=1,  # Just one epoch
    shuffle=False, # Don't shuffle so we can compare to true labels later
    )  # 參考 GitHub/tensorflow-workshop/examples/07_structured_data.ipynb


In [28]:
# 從 wh300/DNNClassifier_Predicting.ipynb 抄來的例子
# Predict 整批 test input 
preds = classifier.predict(pred_input_fn)

In [29]:
# preds 好像是 interator 要反覆研究必須這樣取出來
predict = [i for i in preds];

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model/DNN\model.ckpt-30000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [30]:
# np.argmax() 就是用來從 probabilities array 讀出結論的方法
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.argmax.html

predictions = [np.argmax(predict[i]['probabilities'],axis=-1) for i in range(len(x))]; 
%f ( DNN 猜測的結果 ) predictions tib.
%f ( 跟實際的比較 ) y tib.
%f predictions py> sum(pop()) tib.
%f y py> sum(pop()) tib.
%f predictions np :> array(pop()).shape tib.
%f y np :> array(pop()).shape tib.


( DNN 猜測的結果 ) predictions tib. \ ==> [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [31]:
# 查看 AI 預測的結果，與人工 tag 比較。
categories = [0,0,0,0]
for i in range(len(y)): 
    if int(predictions[i])==int(y[i]) and int(y[i])==0: categories[0]+=1; continue
    if int(predictions[i])!=int(y[i]) and int(y[i])==1: categories[1]+=1; continue
    if int(predictions[i])!=int(y[i]) and int(y[i])==0: categories[2]+=1; continue
    if int(predictions[i])==int(y[i]) and int(y[i])==1: categories[3]+=1; continue
%f categories ( 沒事的，漏掉的，多的，一致的 ) tib.

categories ( 沒事的，漏掉的，多的，一致的 ) tib. \ ==> [194, 156, 0, 0] (<class 'list'>)


### 用 test 資料來比對

In [32]:
# Make input function for prediction:
#   shuffle=False -> do not randomize input data
pred_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": np.array(x_test)[:,:751]},
    y=np.array(y_test),
    num_epochs=1,  # Just one epoch
    shuffle=False, # Don't shuffle so we can compare to true labels later
    )  # 參考 GitHub/tensorflow-workshop/examples/07_structured_data.ipynb


In [33]:
# 從 wh300/DNNClassifier_Predicting.ipynb 抄來的例子
# Predict 整批 test input 
preds = classifier.predict(pred_input_fn)

In [34]:
# preds 好像是 interator 要反覆研究必須這樣取出來
predict = [i for i in preds];

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model/DNN\model.ckpt-30000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [35]:
# np.argmax() 就是用來從 probabilities array 讀出結論的方法
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.argmax.html

predictions = [np.argmax(predict[i]['probabilities'],axis=-1) for i in range(len(x_test))]; 
%f ( DNN 猜測的結果 ) predictions tib.
%f ( 跟實際的比較 ) y_test tib.
%f predictions py> sum(pop()) tib.
%f y_test py> sum(pop()) tib.
%f predictions np :> array(pop()).shape tib.
%f y_test np :> array(pop()).shape tib.


( DNN 猜測的結果 ) predictions tib. \ ==> [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] (<class 'list'>)
( 跟實際的比較 ) y_test tib. \ ==> [1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1] (<class 'list'>)
predictions py> sum(pop()) tib. \ ==> 0 (<class 'numpy.int64'>)
y_test py> sum(pop()) tib. \ ==> 28 (<class 'numpy.int32'>)
predictions np :> array(pop()).shape tib. \ ==> (70,) (<class 'tuple'>)
y_test np :> array(pop()).shape tib. \ ==> (70,) (<class 'tuple'>)


In [36]:
# 查看 AI 預測的結果，與人工 tag 比較。
categories = [0,0,0,0]
for i in range(len(y_test)): 
    if int(predictions[i])==int(y_test[i]) and int(y_test[i])==0: categories[0]+=1; continue
    if int(predictions[i])!=int(y_test[i]) and int(y_test[i])==1: categories[1]+=1; continue
    if int(predictions[i])!=int(y_test[i]) and int(y_test[i])==0: categories[2]+=1; continue
    if int(predictions[i])==int(y_test[i]) and int(y_test[i])==1: categories[3]+=1; continue
%f categories ( 沒事的，漏掉的，多的，一致的 ) tib.

categories ( 沒事的，漏掉的，多的，一致的 ) tib. \ ==> [42, 28, 0, 0] (<class 'list'>)


### 用全部資料來比對

In [ ]:
# Make input function for prediction:
#   shuffle=False -> do not randomize input data
pred_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": np.array(x)[:,:751]},
    y=np.array(y),
    num_epochs=1,  # Just one epoch
    shuffle=False, # Don't shuffle so we can compare to true labels later
    )  # 參考 GitHub/tensorflow-workshop/examples/07_structured_data.ipynb


In [ ]:
# 從 wh300/DNNClassifier_Predicting.ipynb 抄來的例子
# Predict 整批 test input 
preds = classifier.predict(pred_input_fn)

In [ ]:
# preds 好像是 interator 要反覆研究必須這樣取出來
predict = [i for i in preds];

In [ ]:
# np.argmax() 就是用來從 probabilities array 讀出結論的方法
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.argmax.html

predictions = [np.argmax(predict[i]['probabilities'],axis=-1) for i in range(len(x))]; 
%f ( DNN 猜測的結果 ) predictions tib.
%f ( 跟實際的比較 ) y tib.
%f predictions py> sum(pop()) tib.
%f y py> sum(pop()) tib.
%f predictions np :> array(pop()).shape tib.
%f y np :> array(pop()).shape tib.


In [ ]:
# 查看 AI 預測的結果，與人工 tag 比較。
categories = [0,0,0,0]
for i in range(len(y)): 
    if int(predictions[i])==int(y[i]) and int(y[i])==0: categories[0]+=1; continue
    if int(predictions[i])!=int(y[i]) and int(y[i])==1: categories[1]+=1; continue
    if int(predictions[i])!=int(y[i]) and int(y[i])==0: categories[2]+=1; continue
    if int(predictions[i])==int(y[i]) and int(y[i])==1: categories[3]+=1; continue
%f categories ( 沒事的，漏掉的，多的，一致的 ) tib.

In [ ]:
!start tensorboard --logdir="model/DNN"

# 用 iris 來檢驗一下我的程式
###  漲停後 one-shot 天內問【今天續漲嗎？】

iris 就沒問題，效果非常好，那可以說是 data 的問題囉？

In [ ]:
%%js
alert("要不要把 model/DNN directory 先殺掉？")


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import peforth as f
from sklearn import datasets

In [ ]:
%%f
\ Now we redefine the 'unknown' command that does nothing at default  
: unknown ( token -- thing Y|N) // Try to find the unknown token in __main__
  py> getattr(sys.modules['__main__'],pop(),"Ûnknôwn") 
  py> str(tos())=="Ûnknôwn" if drop false else true then ;
  /// here after, when FORTH come accross an unknown token, instead of alerting 
  /// it try to find the token in python __main__ module name space.
: path-to-find-modules ( <path> -- ) // Add path to sys.path so "import module-name" can find the module
    CR word trim ( "path" ) py: sys.path.append(pop()) ;
code \ print(nexttoken('\n')) end-code // Redefine \ command to print the comment line 


In [ ]:
iris = datasets.load_iris()

In [ ]:
x=iris.data 
y=iris.target
%f x :> shape tib.
%f y :> shape tib.    

In [ ]:
%f (  ) x :> shape tib. ( ?, 751 )
%f (  ) x :> shape[0] tib. ( ? )
%f (  ) y :> shape tib. ( ?, )    
%f (  ) y :> shape[0] tib. ( ? )    
%f (  ) y py> sum(pop()) tib. ( 1629 )

In [ ]:
from sklearn import cross_validation
x_train, x_test, y_train, y_test = cross_validation.train_test_split(x, y, test_size = 0.2)

In [ ]:
import sklearn
sklearn.__version__

In [ ]:
%f x_train np :> array(pop()).shape tib.
%f y_train np :> array(pop()).shape tib.
%f x_test np :> array(pop()).shape tib.
%f y_test np :> array(pop()).shape tib.

In [ ]:
# 應該只有一個 feature 是個 shape = (751,) 的 array. 這怎麼表達?
my_feature_columns = [tf.feature_column.numeric_column("x", shape=[4,])]  
my_feature_columns

In [ ]:
# Build 2 hidden layer DNN
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    hidden_units=[64,32],
    n_classes=3,
    dropout=0.2,
    model_dir="model/DNN"
    )

In [ ]:
# Define the training inputs 從 mnist 抄過來改的
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_train},
    y=y_train,
    num_epochs=None,
    batch_size=64,
    shuffle=True
    )

In [ ]:
%pdb off
classifier.train(input_fn=train_input_fn, steps=3000);

In [ ]:
# Define the test inputs  從 mnist 抄過來改的
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_test},
    y=y_test,
    num_epochs=1,
    shuffle=False
    )

In [ ]:
# Evaluate accuracy
accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]
print("\nTest Accuracy: {0:f}%\n".format(accuracy_score*100))

## How about a 'recall' ?

In [ ]:
# Define the test inputs  從 mnist 抄過來改的
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_train},
    y=y_train,
    num_epochs=1,
    shuffle=False
    )

In [ ]:
# Evaluate accuracy
accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]
print("\nTest Accuracy: {0:f}%\n".format(accuracy_score*100))

#  漲停後 one-shot 天內問【今天續漲嗎？】

- 照老師的方法，把資料整理成 150天 一長條一筆，如果往前沒有 150 天就不用。
- 掃出所有漲停的日子，以及後 one-shot 天，如果又有漲停就繼續 one-shot。這樣縮小 dataset 範圍。
- 以此 dataset 問 AI 明天是否【續漲】且【不破前低】？平衡正反雙方，取得 n 值。

In [1]:
%%js
alert("要不要把 model/DNN directory 先殺掉？")
alert("注意 batch_size 合不合理？一千也照跑，結果都是 0.")

<IPython.core.display.Javascript object>

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import peforth as f

In [3]:
%%f
\ Now we redefine the 'unknown' command that does nothing at default  
: unknown ( token -- thing Y|N) // Try to find the unknown token in __main__
  py> getattr(sys.modules['__main__'],pop(),"Ûnknôwn") 
  py> str(tos())=="Ûnknôwn" if drop false else true then ;
  /// here after, when FORTH come accross an unknown token, instead of alerting 
  /// it try to find the token in python __main__ module name space.
: path-to-find-modules ( <path> -- ) // Add path to sys.path so "import module-name" can find the module
    CR word trim ( "path" ) py: sys.path.append(pop()) ;
code \ print(nexttoken('\n')) end-code // Redefine \ command to print the comment line 


reDef unknown
reDef \


In [4]:
data = pd.read_csv('000777.csv',encoding='gbk',parse_dates=[0],index_col=0)

# data frame 中帶有 None 的 rows 都剔除掉。 本來有 4752 rows 現在只剩 4639 
# 用法: df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
data = data.replace(to_replace='None', value=np.nan).dropna()  

# pdb.set_trace() # data 本來是新的在上面
data.sort_index(0,ascending=True,inplace=True) # 排序 sort 過後變成最舊的在上面

# 上市第一天的資料不要，因為【漲跌幅】無效。
data = data.query('index > "1997-07-10"')

In [5]:
# raw data has 20 years long, but here only analysis the last 150 days of a chosen day
dayfeature = 150
oneshot = 5  # 漲停後慣性天數，使正反平衡
dayforesee = oneshot  # 問未來幾天如何如何，最後就要扣掉幾天，避免 out of bounds. 0 代表當天。
featurenum = 5*dayfeature  # 取五個有關係的欄位，ochlq 等五個特徵值。
cross_validation_rate = 0.2
xlength = data.shape[0]-dayfeature-dayforesee

In [6]:
%f ( 總天數 ) data :> shape[0] tib. ( 4638 )
%f ( leading days that are not used ) dayfeature tib.   
%f ( latest days that are not used ) dayforesee tib.
%f ( 有效天數 ) data :> shape[0] dayfeature - dayforesee - tib.

( 總天數 ) data :> shape[0] tib. \ ==> 4638 (<class 'int'>)
( leading days that are not used ) dayfeature tib. \ ==> 150 (<class 'int'>)
( latest days that are not used ) dayforesee tib. \ ==> 5 (<class 'int'>)
( 有效天數 ) data :> shape[0] dayfeature - dayforesee - tib. \ ==> 4483 (<class 'int'>)


In [7]:
x=np.zeros((xlength,featurenum+2))  # +2 是當天開盤跟當天日期
y=np.zeros((xlength));

In [8]:
# 重疊 shift dayfeature (150) 天的五個「行情特徵值」加上當天的開盤 共 751 個參數當作 feature X 
for i in range(0,xlength):
    x[i,0:featurenum]=np.array(data[i:i+dayfeature] \
          [[u'收盘价',u'最高价',u'最低价',u'开盘价',u'成交量']]).reshape((1,featurenum))
    x[i,featurenum]=data.iloc[i+dayfeature][u'开盘价']
    x[i,featurenum+1]=data.index[i+dayfeature].value  # backward pd.to_datetime(value).date()
    

In [9]:
# 預測明天是否會「續漲」且「不破前低」? ochl 
for i in range(0,xlength):
    y[i]=0              
    i0h = float(data.iloc[i][u'最高价']) # i 這天的
    i0l = float(data.iloc[i][u'最低价']) # i 這天的
    i1h = float(data.iloc[i+1][u'最高价']) # i 明天的
    i1l = float(data.iloc[i+1][u'最低价']) # i 明天的
    if i1h > i0h and i1l > i0l :
            y[i]=1
y = np.int32(y)

In [10]:
%f (  ) x :> shape tib. ( ?, 751 )
%f (  ) x :> shape[0] tib. ( ? )
%f (  ) y :> shape tib. ( ?, )    
%f (  ) y :> shape[0] tib. ( ? )    
%f (  ) y py> sum(pop()) tib. ( 1629 )

(  ) x :> shape tib. \ ==> (4483, 752) (<class 'tuple'>)
(  ) x :> shape[0] tib. \ ==> 4483 (<class 'int'>)
(  ) y :> shape tib. \ ==> (4483,) (<class 'tuple'>)
(  ) y :> shape[0] tib. \ ==> 4483 (<class 'int'>)
(  ) y py> sum(pop()) tib. \ ==> 1629 (<class 'numpy.int32'>)


In [11]:
# 直接看【涨跌幅】> 9.8 就算漲停板，從【漲停板】的日子往後多取 oneshot 天。
# 符合條件的日子組成 dataset 其中含有 oneshot 天慣性，使正反兩方平衡。
# 這個簡單的 loop 是挑出要的日子且消除重複的方法，我考慮過用 set() 也是取 unique 的好方法。

x_mask = np.zeros((xlength,)) # 全部放 0 以下只針對目標填 1 
x_mask = np.int32(x_mask)
for i in range(xlength):
    if float(data.iloc[i+dayfeature-1][u'涨跌幅'])>=9.8:
        for j in range(oneshot):
            x_mask[i+j-1] = 1

In [12]:
%f x_mask tib.
%f x_mask :> shape tib.
%f x_mask py> sum(pop()) tib.

x_mask tib. \ ==> [0 0 0 ..., 0 0 0] (<class 'numpy.ndarray'>)
x_mask :> shape tib. \ ==> (4483,) (<class 'tuple'>)
x_mask py> sum(pop()) tib. \ ==> 350 (<class 'numpy.int32'>)


In [13]:
x_original = []
y_original = []
for i in range(xlength):
    if x_mask[i]:
        x_original.append(x[i])
        y_original.append(y[i])
x,x_original = x_original,x
y,y_original = y_original,y

In [14]:
%f x_original np :> array(pop()).shape tib.
%f y_original np :> array(pop()).shape tib.
%f x np :> array(pop()).shape tib.
%f y np :> array(pop()).shape tib.
%f ( 差不多是 dataset 總數的一半嗎？ ) y py> sum(pop()) tib.

x_original np :> array(pop()).shape tib. \ ==> (4483, 752) (<class 'tuple'>)
y_original np :> array(pop()).shape tib. \ ==> (4483,) (<class 'tuple'>)
x np :> array(pop()).shape tib. \ ==> (350, 752) (<class 'tuple'>)
y np :> array(pop()).shape tib. \ ==> (350,) (<class 'tuple'>)
( 差不多是 dataset 總數的一半嗎？ ) y py> sum(pop()) tib. \ ==> 156 (<class 'numpy.int32'>)


In [15]:
from sklearn import cross_validation
x_train, x_test, y_train, y_test = cross_validation.train_test_split(x, y, test_size = 0.2)

c:\users\hcche\appdata\local\programs\python\python36\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [16]:
import sklearn
sklearn.__version__

'0.19.1'

In [17]:
%f x_train np :> array(pop()).shape tib.
%f y_train np :> array(pop()).shape tib.
%f x_test np :> array(pop()).shape tib.
%f y_test np :> array(pop()).shape tib.
%f ( numpy 才能裁切 matrix ) x_train np :> array(pop())[:,:751].shape tib.
%f ( numpy 才能裁切 matrix ) x_test np :> array(pop())[:,:751].shape tib.


x_train np :> array(pop()).shape tib. \ ==> (280, 752) (<class 'tuple'>)
y_train np :> array(pop()).shape tib. \ ==> (280,) (<class 'tuple'>)
x_test np :> array(pop()).shape tib. \ ==> (70, 752) (<class 'tuple'>)
y_test np :> array(pop()).shape tib. \ ==> (70,) (<class 'tuple'>)
( numpy 才能裁切 matrix ) x_train np :> array(pop())[:,:751].shape tib. \ ==> (280, 751) (<class 'tuple'>)
( numpy 才能裁切 matrix ) x_test np :> array(pop())[:,:751].shape tib. \ ==> (70, 751) (<class 'tuple'>)


In [18]:
# 應該只有一個 feature 是個 shape = (751,) 的 array. 這怎麼表達?
my_feature_columns = [tf.feature_column.numeric_column("x", shape=[751,])]  
my_feature_columns

[_NumericColumn(key='x', shape=(751,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [19]:
# Build 2 hidden layer DNN with 10, 10 units respectively.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 10 nodes each.
    hidden_units=[64,32],
    # The model must choose between 3 classes.
    n_classes=2,
    dropout=0.2,
    model_dir="model/DNN"
    )

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'model/DNN', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000022621116BE0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [20]:
# Define the training inputs 從 mnist 抄過來改的
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": np.array(x_train)[:,:751]},
    y=np.array(y_train),
    num_epochs=None,
    batch_size=64,
    shuffle=True
    )

In [21]:
%pdb off
classifier.train(input_fn=train_input_fn, steps=30000);

Automatic pdb calling has been turned OFF
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into model/DNN\model.ckpt.
INFO:tensorflow:loss = 2.50763e+08, step = 1
INFO:tensorflow:global_step/sec: 334.799
INFO:tensorflow:loss = 141593.0, step = 101 (0.299 sec)
INFO:tensorflow:global_step/sec: 418.432
INFO:tensorflow:loss = 20543.8, step = 201 (0.243 sec)
INFO:tensorflow:global_step/sec: 474.202
INFO:tensorflow:loss = 2006.95, step = 301 (0.210 sec)
INFO:tensorflow:global_step/sec: 474.208
INFO:tensorflow:loss = 1642.81, step = 401 (0.216 sec)
INFO:tensorflow:global_step/sec: 454.806
INFO:tensorflow:loss = 2754.9, step = 501 (0.216 sec)
INFO:tensorflow:global_step/sec: 448.688
INFO:tensorflow:loss = 2314.37, step = 601 (0.222 sec)
INFO:tensorflow:global_step/sec:

INFO:tensorflow:global_step/sec: 427.593
INFO:tensorflow:loss = 99.6241, step = 8201 (0.235 sec)
INFO:tensorflow:global_step/sec: 458.981
INFO:tensorflow:loss = 43.2217, step = 8301 (0.217 sec)
INFO:tensorflow:global_step/sec: 413.457
INFO:tensorflow:loss = 43.9058, step = 8401 (0.244 sec)
INFO:tensorflow:global_step/sec: 365.175
INFO:tensorflow:loss = 56.7049, step = 8501 (0.274 sec)
INFO:tensorflow:global_step/sec: 436.93
INFO:tensorflow:loss = 43.6482, step = 8601 (0.228 sec)
INFO:tensorflow:global_step/sec: 465.384
INFO:tensorflow:loss = 44.0812, step = 8701 (0.216 sec)
INFO:tensorflow:global_step/sec: 431.28
INFO:tensorflow:loss = 43.9111, step = 8801 (0.229 sec)
INFO:tensorflow:global_step/sec: 444.699
INFO:tensorflow:loss = 42.6944, step = 8901 (0.226 sec)
INFO:tensorflow:global_step/sec: 433.149
INFO:tensorflow:loss = 43.5813, step = 9001 (0.232 sec)
INFO:tensorflow:global_step/sec: 450.709
INFO:tensorflow:loss = 131.365, step = 9101 (0.221 sec)
INFO:tensorflow:global_step/sec:

INFO:tensorflow:global_step/sec: 406.65
INFO:tensorflow:loss = 13588.2, step = 16601 (0.232 sec)
INFO:tensorflow:global_step/sec: 491.23
INFO:tensorflow:loss = 43.2174, step = 16701 (0.210 sec)
INFO:tensorflow:global_step/sec: 434.042
INFO:tensorflow:loss = 6599.68, step = 16801 (0.222 sec)
INFO:tensorflow:global_step/sec: 491.785
INFO:tensorflow:loss = 294.223, step = 16901 (0.220 sec)
INFO:tensorflow:global_step/sec: 431.183
INFO:tensorflow:loss = 43.7396, step = 17001 (0.212 sec)
INFO:tensorflow:global_step/sec: 378.262
INFO:tensorflow:loss = 43.4706, step = 17101 (0.268 sec)
INFO:tensorflow:global_step/sec: 450.711
INFO:tensorflow:loss = 43.9169, step = 17201 (0.221 sec)
INFO:tensorflow:global_step/sec: 452.749
INFO:tensorflow:loss = 43.0679, step = 17301 (0.221 sec)
INFO:tensorflow:global_step/sec: 408.891
INFO:tensorflow:loss = 43.0593, step = 17401 (0.245 sec)
INFO:tensorflow:global_step/sec: 460.499
INFO:tensorflow:loss = 43.9119, step = 17501 (0.213 sec)
INFO:tensorflow:global

INFO:tensorflow:global_step/sec: 401.838
INFO:tensorflow:loss = 44.0829, step = 25001 (0.248 sec)
INFO:tensorflow:global_step/sec: 469.604
INFO:tensorflow:loss = 43.0611, step = 25101 (0.210 sec)
INFO:tensorflow:global_step/sec: 424.114
INFO:tensorflow:loss = 43.5748, step = 25201 (0.236 sec)
INFO:tensorflow:global_step/sec: 403.07
INFO:tensorflow:loss = 44.4865, step = 25301 (0.257 sec)
INFO:tensorflow:global_step/sec: 462.217
INFO:tensorflow:loss = 44.5886, step = 25401 (0.207 sec)
INFO:tensorflow:global_step/sec: 423.463
INFO:tensorflow:loss = 72.6502, step = 25501 (0.252 sec)
INFO:tensorflow:global_step/sec: 432.073
INFO:tensorflow:loss = 43.912, step = 25601 (0.216 sec)
INFO:tensorflow:global_step/sec: 411.762
INFO:tensorflow:loss = 44.0823, step = 25701 (0.248 sec)
INFO:tensorflow:global_step/sec: 442.496
INFO:tensorflow:loss = 43.5707, step = 25801 (0.221 sec)
INFO:tensorflow:global_step/sec: 411.192
INFO:tensorflow:loss = 264.303, step = 25901 (0.246 sec)
INFO:tensorflow:global

In [22]:
# Define the test inputs  從 mnist 抄過來改的
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": np.array(x_test)[:,:751]},
    y=np.array(y_test),
    num_epochs=1,
    shuffle=False
    )

In [23]:
# Evaluate accuracy
evaluate = classifier.evaluate(input_fn=test_input_fn)
accuracy_score = evaluate["accuracy"]
print("\nTest Accuracy: {0:f}%\n".format(accuracy_score*100))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-06-09:52:13
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model/DNN\model.ckpt-30000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-06-06-09:52:14
INFO:tensorflow:Saving dict for global step 30000: accuracy = 0.6, accuracy_baseline = 0.6, auc = 0.5, auc_precision_recall = 0.7, average_loss = 0.67971, global_step = 30000, label/mean = 0.4, loss = 47.5797, precision = 0.0, prediction/mean = 0.457405, recall = 0.0

Test Accuracy: 60.000002%



In [24]:
evaluate

{'accuracy': 0.60000002,
 'accuracy_baseline': 0.60000002,
 'auc': 0.5,
 'auc_precision_recall': 0.69999999,
 'average_loss': 0.67970961,
 'global_step': 30000,
 'label/mean': 0.40000001,
 'loss': 47.579674,
 'precision': 0.0,
 'prediction/mean': 0.45740461,
 'recall': 0.0}

## How about a 'recall' ?

In [25]:
# Define the test inputs  從 mnist 抄過來改的
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": np.array(x_train)[:,:751]},
    y=np.array(y_train),
    num_epochs=1,
    shuffle=False
    )

In [26]:
# Evaluate accuracy
accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]
print("\nTest Accuracy: {0:f}%\n".format(accuracy_score*100))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-06-09:52:17
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model/DNN\model.ckpt-30000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-06-06-09:52:18
INFO:tensorflow:Saving dict for global step 30000: accuracy = 0.542857, accuracy_baseline = 0.542857, auc = 0.5, auc_precision_recall = 0.728571, average_loss = 0.689469, global_step = 30000, label/mean = 0.457143, loss = 64.3505, precision = 0.0, prediction/mean = 0.457405, recall = 0.0

Test Accuracy: 54.285717%



### 用全部資料來比對

In [27]:
# Make input function for prediction:
#   shuffle=False -> do not randomize input data
pred_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": np.array(x)[:,:751]},
    y=np.array(y),
    num_epochs=1,  # Just one epoch
    shuffle=False, # Don't shuffle so we can compare to true labels later
    )  # 參考 GitHub/tensorflow-workshop/examples/07_structured_data.ipynb


In [28]:
# 從 wh300/DNNClassifier_Predicting.ipynb 抄來的例子
# Predict 整批 test input 
preds = classifier.predict(pred_input_fn)

In [29]:
# preds 好像是 interator 要反覆研究必須這樣取出來
predict = [i for i in preds];

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model/DNN\model.ckpt-30000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [30]:
# np.argmax() 就是用來從 probabilities array 讀出結論的方法
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.argmax.html

predictions = [np.argmax(predict[i]['probabilities'],axis=-1) for i in range(len(x))]; 
%f ( DNN 猜測的結果 ) predictions tib.
%f ( 跟實際的比較 ) y tib.
%f predictions py> sum(pop()) tib.
%f y py> sum(pop()) tib.
%f predictions np :> array(pop()).shape tib.
%f y np :> array(pop()).shape tib.


( DNN 猜測的結果 ) predictions tib. \ ==> [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [31]:
# 查看 AI 預測的結果，與人工 tag 比較。
categories = [0,0,0,0]
for i in range(len(y)): 
    if int(predictions[i])==int(y[i]) and int(y[i])==0: categories[0]+=1; continue
    if int(predictions[i])!=int(y[i]) and int(y[i])==1: categories[1]+=1; continue
    if int(predictions[i])!=int(y[i]) and int(y[i])==0: categories[2]+=1; continue
    if int(predictions[i])==int(y[i]) and int(y[i])==1: categories[3]+=1; continue
%f categories ( 沒事的，漏掉的，多的，一致的 ) tib.

categories ( 沒事的，漏掉的，多的，一致的 ) tib. \ ==> [194, 156, 0, 0] (<class 'list'>)


### 用 test 資料來比對

In [32]:
# Make input function for prediction:
#   shuffle=False -> do not randomize input data
pred_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": np.array(x_test)[:,:751]},
    y=np.array(y_test),
    num_epochs=1,  # Just one epoch
    shuffle=False, # Don't shuffle so we can compare to true labels later
    )  # 參考 GitHub/tensorflow-workshop/examples/07_structured_data.ipynb


In [33]:
# 從 wh300/DNNClassifier_Predicting.ipynb 抄來的例子
# Predict 整批 test input 
preds = classifier.predict(pred_input_fn)

In [34]:
# preds 好像是 interator 要反覆研究必須這樣取出來
predict = [i for i in preds];

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model/DNN\model.ckpt-30000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [35]:
# np.argmax() 就是用來從 probabilities array 讀出結論的方法
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.argmax.html

predictions = [np.argmax(predict[i]['probabilities'],axis=-1) for i in range(len(x_test))]; 
%f ( DNN 猜測的結果 ) predictions tib.
%f ( 跟實際的比較 ) y_test tib.
%f predictions py> sum(pop()) tib.
%f y_test py> sum(pop()) tib.
%f predictions np :> array(pop()).shape tib.
%f y_test np :> array(pop()).shape tib.


( DNN 猜測的結果 ) predictions tib. \ ==> [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] (<class 'list'>)
( 跟實際的比較 ) y_test tib. \ ==> [1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1] (<class 'list'>)
predictions py> sum(pop()) tib. \ ==> 0 (<class 'numpy.int64'>)
y_test py> sum(pop()) tib. \ ==> 28 (<class 'numpy.int32'>)
predictions np :> array(pop()).shape tib. \ ==> (70,) (<class 'tuple'>)
y_test np :> array(pop()).shape tib. \ ==> (70,) (<class 'tuple'>)


In [36]:
# 查看 AI 預測的結果，與人工 tag 比較。
categories = [0,0,0,0]
for i in range(len(y_test)): 
    if int(predictions[i])==int(y_test[i]) and int(y_test[i])==0: categories[0]+=1; continue
    if int(predictions[i])!=int(y_test[i]) and int(y_test[i])==1: categories[1]+=1; continue
    if int(predictions[i])!=int(y_test[i]) and int(y_test[i])==0: categories[2]+=1; continue
    if int(predictions[i])==int(y_test[i]) and int(y_test[i])==1: categories[3]+=1; continue
%f categories ( 沒事的，漏掉的，多的，一致的 ) tib.

categories ( 沒事的，漏掉的，多的，一致的 ) tib. \ ==> [42, 28, 0, 0] (<class 'list'>)


### 用全部資料來比對

In [ ]:
# Make input function for prediction:
#   shuffle=False -> do not randomize input data
pred_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": np.array(x)[:,:751]},
    y=np.array(y),
    num_epochs=1,  # Just one epoch
    shuffle=False, # Don't shuffle so we can compare to true labels later
    )  # 參考 GitHub/tensorflow-workshop/examples/07_structured_data.ipynb


In [ ]:
# 從 wh300/DNNClassifier_Predicting.ipynb 抄來的例子
# Predict 整批 test input 
preds = classifier.predict(pred_input_fn)

In [ ]:
# preds 好像是 interator 要反覆研究必須這樣取出來
predict = [i for i in preds];

In [ ]:
# np.argmax() 就是用來從 probabilities array 讀出結論的方法
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.argmax.html

predictions = [np.argmax(predict[i]['probabilities'],axis=-1) for i in range(len(x))]; 
%f ( DNN 猜測的結果 ) predictions tib.
%f ( 跟實際的比較 ) y tib.
%f predictions py> sum(pop()) tib.
%f y py> sum(pop()) tib.
%f predictions np :> array(pop()).shape tib.
%f y np :> array(pop()).shape tib.


In [ ]:
# 查看 AI 預測的結果，與人工 tag 比較。
categories = [0,0,0,0]
for i in range(len(y)): 
    if int(predictions[i])==int(y[i]) and int(y[i])==0: categories[0]+=1; continue
    if int(predictions[i])!=int(y[i]) and int(y[i])==1: categories[1]+=1; continue
    if int(predictions[i])!=int(y[i]) and int(y[i])==0: categories[2]+=1; continue
    if int(predictions[i])==int(y[i]) and int(y[i])==1: categories[3]+=1; continue
%f categories ( 沒事的，漏掉的，多的，一致的 ) tib.

In [39]:
np.array(x_test)[:10,751]

array([  1.47432960e+18,   1.16959680e+18,   9.30873600e+17,
         1.43337600e+18,   1.41168960e+18,   9.29491200e+17,
         1.30515840e+18,   1.25383680e+18,   1.02522240e+18,
         1.17987840e+18])

In [41]:
pd.to_datetime(np.array(x_test)[:10,751])


DatetimeIndex(['2016-09-20', '2007-01-24', '1999-07-02', '2015-06-04',
               '2014-09-26', '1999-06-16', '2011-05-12', '2009-09-25',
               '2002-06-28', '2007-05-23'],
              dtype='datetime64[ns]', freq=None)

In [ ]:
!start tensorboard --logdir="model/DNN"

#  [待續] 改良餵進資料的方式，直接用 shape(150,5) 跳過 150天打包的過程。

- 要回溯的天數先指定，最前面這麼多天扣掉。
- 要往後參考的天數先指定，最後面這麼多天扣掉。排除這兩條件，剩下來的就是可用的天數。
- cross_validation 的工作很簡單，看取多少趴數當 test 從尾巴切走就對了。所以 ％ 數對應一個切分日期。
- 改寫 input_fn 讓它從 train or test set 裡面挑一個出來

# 預測「未來 dayforesee 天會不會漲 20%？」

實際 4638天當中有 1677天 符合條件
用全部資料來比對： ( 沒事的，漏掉的，多的，一致的 ) tib. \ ==> [2418, 1243, 383, 434] 
用 Test 比對 ： ( 沒事的，漏掉的，多的，一致的 ) tib. \ ==> [433, 297, 110, 56]
這樣看起來好像有點被記住了的問題，所以 dropout 用到了 0.4 （40％） 得到以上結果，只能這樣了。


In [ ]:
%%js
alert("要不要把 model/DNN directory 先殺掉？")


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import peforth as f
from sklearn import cross_validation

In [ ]:
%%f
\ Now we redefine the 'unknown' command that does nothing at default  
: unknown ( token -- thing Y|N) // Try to find the unknown token in __main__
  py> getattr(sys.modules['__main__'],pop(),"Ûnknôwn") 
  py> str(tos())=="Ûnknôwn" if drop false else true then ;
  /// here after, when FORTH come accross an unknown token, instead of alerting 
  /// it try to find the token in python __main__ module name space.
: path-to-find-modules ( <path> -- ) // Add path to sys.path so "import module-name" can find the module
    CR word trim ( "path" ) py: sys.path.append(pop()) ;
code \ print(nexttoken('\n')) end-code // Redefine \ command to print the comment line 


In [ ]:
data = pd.read_csv('000777.csv',encoding='gbk',parse_dates=[0],index_col=0)

# data frame 中帶有 None 的 rows 都剔除掉。 本來有 4752 rows 現在只剩 4639 
# 用法: df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
data = data.replace(to_replace='None', value=np.nan).dropna()  

# pdb.set_trace() # data 本來是新的在上面
data.sort_index(0,ascending=True,inplace=True) # 排序 sort 過後變成最舊的在上面

# 上市第一天的資料不要，因為【漲跌幅】無效。
data = data.query('index > "1997-07-10"')

In [ ]:
# raw data has 20 years long, but here only analysis the last 150 days of a chosen day
dayfeature=150
dayforesee=10  # 問未來幾天如何如何，最後就要扣掉幾天，避免 out of bounds. 0 代表當天。
featurenum=5*dayfeature  # 取五個有關係的欄位，特徵值。
x=np.zeros((data.shape[0]-dayfeature-dayforesee,featurenum+1))
y=np.zeros((data.shape[0]-dayfeature-dayforesee));

In [ ]:
# 重疊 shift 150 天的五個「行情數目」加上下一天的開盤 共 751 個「數目」當作 feature X 

for i in range(0,data.shape[0]-dayfeature-dayforesee):
    x[i,0:featurenum]=np.array(data[i:i+dayfeature] \
          [[u'收盘价',u'最高价',u'最低价',u'开盘价',u'成交量']]).reshape((1,featurenum))
    # x[i,featurenum]=data.ix[i+dayfeature][u'开盘价']  # ix deprecated
    x[i,featurenum]=data.iloc[i+dayfeature][u'开盘价']

In [ ]:
# 預測未來 dayforesee 天是否會「漲20%」?

for i in range(0,data.shape[0]-dayfeature-dayforesee):
    y[i]=0              
    pi = float(data.iloc[i][u'收盘价'])
    for j in range(dayforesee):
        pi5 = float(data.iloc[j+i+dayfeature][u'最高价'])
        if (pi5-pi)/pi >= 0.20 :
            y[i]=1
y = np.int32(y)

In [ ]:
x_train, x_test, y_train, y_test = cross_validation.train_test_split(x, y, test_size = 0.2)

In [ ]:
%f ( 有這麼多天符合條件 ) y py> sum(pop()) tib.
%f x_train :> shape tib.
%f x_test :> shape tib.
%f y_train :> shape tib.
%f y_test :> shape tib.
%f x_train :> shape[0] tib.
    

In [ ]:
y_train

In [ ]:
# 應該只有一個 feature 是個 shape = (751,) 的 array. 這怎麼表達?
my_feature_columns = [tf.feature_column.numeric_column("x", shape=[751,])]  
my_feature_columns

In [ ]:
# Build 2 hidden layer DNN with 10, 10 units respectively.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 10 nodes each.
    hidden_units=[256,128],
    # The model must choose between 3 classes.
    n_classes=2,
    dropout=0.4,
    model_dir="model/DNN"
    )

In [ ]:
# Define the training inputs 從 mnist 抄過來改的
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_train},
    y=y_train,
    num_epochs=None,
    batch_size=1000,
    shuffle=True
    )

In [ ]:
classifier.train(input_fn=train_input_fn, steps=3000);

In [ ]:
# Define the test inputs  從 mnist 抄過來改的
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_test},
    y=y_test,
    num_epochs=1,
    shuffle=False
    )

In [ ]:
# Evaluate accuracy
accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]
print("\nTest Accuracy: {0:f}%\n".format(accuracy_score*100))

# End of "mnist_estimator.py"


### 用全部資料來比對

In [ ]:
# Make input function for prediction:
#   shuffle=False -> do not randomize input data
pred_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x},
    y=y,
    num_epochs=1,  # Just one epoch
    shuffle=False, # Don't shuffle so we can compare to true labels later
    )  # 參考 GitHub/tensorflow-workshop/examples/07_structured_data.ipynb


In [ ]:
# 從 wh300/DNNClassifier_Predicting.ipynb 抄來的例子
# Predict 整批 test input 
preds = classifier.predict(pred_input_fn)

In [ ]:
predict = [i for i in preds];

In [ ]:
predictions = [int(i['probabilities'][1] > i['probabilities'][0]) for i in predict];

In [ ]:
%f predictions py> sum(pop()) tib.
%f y py> sum(pop()) tib.
%f y :> [:50] . cr


In [ ]:
# 查看 AI 預測的結果，與人工 tag 比較。
categories = [0,0,0,0]
for i in range(y.shape[0]): 
    if int(predictions[i])==int(y[i]) and int(y[i])==0: categories[0]+=1; continue
    if int(predictions[i])!=int(y[i]) and int(y[i])==1: categories[1]+=1; continue
    if int(predictions[i])!=int(y[i]) and int(y[i])==0: categories[2]+=1; continue
    if int(predictions[i])==int(y[i]) and int(y[i])==1: categories[3]+=1; continue
%f categories ( 沒事的，漏掉的，多的，一致的 ) tib.

### 只用 Test dataset 來比對

In [ ]:
# Make input function for prediction:
#   shuffle=False -> do not randomize input data
pred_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_test},
    y=y_test,
    num_epochs=1,  # Just one epoch
    shuffle=False, # Don't shuffle so we can compare to true labels later
    )  # 參考 GitHub/tensorflow-workshop/examples/07_structured_data.ipynb


In [ ]:
# 從 wh300/DNNClassifier_Predicting.ipynb 抄來的例子
# Predict 整批 test input 
preds = classifier.predict(pred_input_fn)

In [ ]:
predict = [i for i in preds];

In [ ]:
predictions = [int(i['probabilities'][1] > i['probabilities'][0]) for i in predict];

In [ ]:
%f predictions py> sum(pop()) tib.
%f y py> sum(pop()) tib.
%f y :> [:50] . cr


In [ ]:
# 查看 AI 預測的結果，與人工 tag 比較。
categories = [0,0,0,0]
for i in range(y_test.shape[0]): 
    if int(predictions[i])==int(y[i]) and int(y[i])==0: categories[0]+=1; continue
    if int(predictions[i])!=int(y[i]) and int(y[i])==1: categories[1]+=1; continue
    if int(predictions[i])!=int(y[i]) and int(y[i])==0: categories[2]+=1; continue
    if int(predictions[i])==int(y[i]) and int(y[i])==1: categories[3]+=1; continue
%f categories ( 沒事的，漏掉的，多的，一致的 ) tib.

In [ ]:
!start tensorboard --logdir="model/DNN"

# 都搞懂了，接下來打包 000777.csv 試試看吧!

這把預測「明天會不會漲停？」結果因為實際 4638天當中只有88天漲停，全部猜「不」就有 98.10％ 的 accuracy. 可能正是因為怎麼猜別的都沒有這個好。 DNN 全部猜「不會」。


In [ ]:
%%js
alert("要不要把 DNN_model directory 先殺掉？")


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import peforth as f
from sklearn import cross_validation

In [ ]:
%%f
\ Now we redefine the 'unknown' command that does nothing at default  
: unknown ( token -- thing Y|N) // Try to find the unknown token in __main__
  py> getattr(sys.modules['__main__'],pop(),"Ûnknôwn") 
  py> str(tos())=="Ûnknôwn" if drop false else true then ;
  /// here after, when FORTH come accross an unknown token, instead of alerting 
  /// it try to find the token in python __main__ module name space.
: path-to-find-modules ( <path> -- ) // Add path to sys.path so "import module-name" can find the module
    CR word trim ( "path" ) py: sys.path.append(pop()) ;
code \ print(nexttoken('\n')) end-code // Redefine \ command to print the comment line 


In [ ]:
data = pd.read_csv('000777.csv',encoding='gbk',parse_dates=[0],index_col=0)

# data frame 中帶有 None 的 rows 都剔除掉。 本來有 4752 rows 現在只剩 4639 
# 用法: df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
data = data.replace(to_replace='None', value=np.nan).dropna()  

# pdb.set_trace() # data 本來是新的在上面
data.sort_index(0,ascending=True,inplace=True) # 排序 sort 過後變成最舊的在上面

# 上市第一天的資料不要，因為【漲跌幅】無效。
data = data.query('index > "1997-07-10"')

In [ ]:
# raw data has 20 years long, but here only analysis the last 150 days of a chosen day
dayfeature=150
dayforesee=5  # 問未來幾天如何如何，最後就要扣掉幾天，避免 out of bounds. 0 代表當天。
featurenum=5*dayfeature  # 取五個有關係的欄位，特徵值。
x=np.zeros((data.shape[0]-dayfeature-dayforesee,featurenum+1))
y=np.zeros((data.shape[0]-dayfeature-dayforesee));

In [ ]:
data.shape

In [ ]:
# 重疊 shift 150 天的五個「行情數目」加上下一天的開盤 共 751 個「數目」當作 feature X 

for i in range(0,data.shape[0]-dayfeature-dayforesee):
    x[i,0:featurenum]=np.array(data[i:i+dayfeature] \
          [[u'收盘价',u'最高价',u'最低价',u'开盘价',u'成交量']]).reshape((1,featurenum))
    # x[i,featurenum]=data.ix[i+dayfeature][u'开盘价']  # ix deprecated
    x[i,featurenum]=data.iloc[i+dayfeature][u'开盘价']

In [ ]:
# 預測當天是否會「漲停」?

# data.shape[0]-dayfeature 是 4489 同上 x, 所以這是在製作相對所有 x 的 y. x 是 (4488, 751)
# dayfeature 是 150 
# 直接看【涨跌幅】> 9.8 就算漲停板

for i in range(0,data.shape[0]-dayfeature-dayforesee):
    if float(data.iloc[i+dayfeature][u'涨跌幅'])>=9.8:
        y[i]=1.0
    else:
        y[i]=0.0          

In [ ]:
x_train, x_test, y_train, y_test = cross_validation.train_test_split(x, y, test_size = 0.2)

In [ ]:
%f x_train :> shape tib.
%f x_test :> shape tib.
%f y_train :> shape tib.
%f y_test :> shape tib.
%f x_train :> shape[0] tib.
    

用 cross_validation 切分好 train-test feature & label 之後, 可望弄成 input_fn 要的形式

In [ ]:
# 應該只有一個 feature 是個 shape = (751,) 的 array. 這怎麼表達?
my_feature_columns = [tf.feature_column.numeric_column("x", shape=[751,])]  
my_feature_columns

In [ ]:
# Build 2 hidden layer DNN with 10, 10 units respectively.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 10 nodes each.
    hidden_units=[256, 128],
    # The model must choose between 3 classes.
    n_classes=2,
    dropout=0.2,
    model_dir="DNN_model"
    )

以上 classifier 弄出來了，還是要面對 input_function 怎麼弄的問題。Lambda 太難懂了，從 wh300 或哪裡去抄吧。
看到的都是 tf.estimator.inputs.pandas_input_fn 我曾查出有 'numpy_input_fn' 

    c:\Users\hcche\Documents\GitHub\twstock\mnist_estimator.ipynb 
    # Specify feature
    feature_columns = [tf.feature_column.numeric_column("x", shape=[28, 28])]
    幹！ 回想起來了，用這個例子可以簡化 150天 的資料製作方式

Estimator 的 DNNClassifier 怎麼用? 上面這個 mnist 也有似乎更進一步的發揮

    # Build 2 layer DNN classifier
    classifier = tf.estimator.DNNClassifier(
        feature_columns=feature_columns,
        hidden_units=[256, 32],
        optimizer=tf.train.AdamOptimizer(1e-4),
        n_classes=10,
        dropout=0.1,
        model_dir="./tmp/mnist_model"
        )
    

In [ ]:
# Define the training inputs 從 mnist 抄過來改的
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_train},
    y=y_train,
    num_epochs=None,
    batch_size=x_train.shape[0],  # 漲停很少，故 batch 要用全部天數，否則它全部猜「不會」
    shuffle=True
    )

In [ ]:
classifier.train(input_fn=train_input_fn, steps=1000);

In [ ]:
# Define the test inputs  從 mnist 抄過來改的
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_test},
    y=y_test,
    num_epochs=1,
    shuffle=False
    )

In [ ]:
# Evaluate accuracy
accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]
print("\nTest Accuracy: {0:f}%\n".format(accuracy_score*100))

# End of "mnist_estimator.py"


讚！ 接下來怎麼看 predict 的結果 . . . 參考 wh300 吧


In [ ]:
# Make input function for prediction:
#   shuffle=False -> do not randomize input data
pred_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x},
    y=y,
    num_epochs=1,  # Just one epoch
    shuffle=False, # Don't shuffle so we can compare to true labels later
    )  # 參考 GitHub/tensorflow-workshop/examples/07_structured_data.ipynb


In [ ]:
# 從 wh300/DNNClassifier_Predicting.ipynb 抄來的例子
# Predict 整批 test input 
preds = classifier.predict(pred_input_fn)

In [ ]:
predictions = [int(i['probabilities'][1] > i['probabilities'][0]) for i in preds];

In [ ]:
%f predictions py> sum(pop()) tib.
%f y py> sum(pop()) tib.


# 我在 Tensorflow 官網的教學網頁上找到了用 Iris 當例題的 estimator tutorial 

https://www.tensorflow.org/versions/master/get_started/premade_estimators (also on my Ynote "Get Started with Estimators _TensorFlow_") 有正式、詳細的介紹。
這篇文章鼓勵人用 Estimator API 跟 Dataset API。本文研讀、實驗這篇文章。
文章引用的 GitHub resource 我已經 clone 下來了在 T550 `c:\Users\hcche\Documents\GitHub\models`  

重點是，直接用 pandas data-frame 來當作 feature - label 餵給 input function 這樣更自然，免去搞 150 天行情資料的麻煩。


In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

In [ ]:
from sklearn import datasets
import numpy as np
import pandas as pd

In [ ]:
import peforth

In [ ]:
%%f
\ Now we redefine the 'unknown' command that does nothing at default  
: unknown ( token -- thing Y|N) // Try to find the unknown token in __main__
  py> getattr(sys.modules['__main__'],pop(),"Ûnknôwn") 
  py> str(tos())=="Ûnknôwn" if drop false else true then ;
  /// here after, when FORTH come accross an unknown token, instead of alerting 
  /// it try to find the token in python __main__ module name space.

: path-to-find-modules ( <path> -- ) // Add path to sys.path so "import module-name" can find the module
    CR word trim ( "path" ) py: sys.path.append(pop()) ;

\ set path for 'import iris_data'
path-to-find-modules c:\Users\hcche\Documents\GitHub\models\samples\core\get_started

code \ print(nexttoken('\n')) end-code // Redefine \ command to print the comment line 


In [ ]:
iris = datasets.load_iris()

In [ ]:
iris.feature_names

In [ ]:
iris.data[:4]

In [ ]:
iris.data.shape

In [ ]:
# df = pd.DataFrame(np.array(irisX).reshape(-1,4), columns = iris.feature_names)    
x = pd.DataFrame(iris.data, columns = iris.feature_names)    
x[:4]

In [ ]:
y = pd.DataFrame(iris.target, columns = ["target"])    
y[48:52]

In [ ]:
# https://www.tensorflow.org/api_docs/python/tf/estimator/inputs/pandas_input_fn

tf.estimator.inputs.pandas_input_fn(
    x,
    y,
    batch_size=64,
    num_epochs=5,
    shuffle=False,
    queue_capacity=None,
    num_threads=1,
    target_column='target'
)

# 照 Tutorial 執行 -- 一把就成功！

直接把本 notebook 建個 hard link 到 c:\Users\hcche\Documents\GitHub\models\samples\core\get_started 去執行...


In [ ]:
%run premade_estimator.py

## 開始照教材研讀
    
premade_estimator.py iris_data.py 都抓過來，他本來都放在 main() 內執行，不利 study 今予拆開，分段執行亦可，如下：

In [ ]:
#  Copyright 2016 The TensorFlow Authors. All Rights Reserved.
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
"""An Example of a DNNClassifier for the Iris dataset."""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import tensorflow as tf

import iris_data  # iris_data.py 在本目錄下 GitHub\models\samples\core\get_started\iris_data.py 

這 argparse module 很常見 DeepSpeech 就有，這裡用不著吧！

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--batch_size', default=100, type=int, help='batch size')
parser.add_argument('--train_steps', default=1000, type=int,
                    help='number of training steps')

In [ ]:
# 不能在 jupyter notebook 下直接跑，因為 argv 不如預期。
from IPython.display import display,Image;display(Image('error1.jpg', width=700))

#### Command Line Arguments 要用模擬的
Module argparse 只能在 python command line 跑

In [ ]:
%f \ [x] 沒有指定時，自動取 default 值，真不知道這哪來的？看到了，從 parser 那邊 code 裡給的。
args = parser.parse_args(["--batch_size=100", "--train_steps=1000"])
args

#### 研究  (train_x, train_y), (test_x, test_y) = iris_data.load_data()

看出來了 iris_data.py 是把 150 筆資料切成 120 : 30 for training and testing respectively.
這是個壞消息，他沒有像 sklearn 的自動切的那啥 function. 

In [ ]:
 (train_x, train_y), (test_x, test_y) = iris_data.load_data()

In [ ]:
%f train_x type . cr
train_x

In [ ]:
%f train_y type . cr
train_y

In [ ]:
%f test_x type . cr
test_x

In [ ]:
%f test_y type . cr
test_y

In [ ]:
%f \ [ ] 這是 feature columns, 我晚點可以嘗試把它弄成光一個 shape 為 (4,) 的單一 feature 
for key in train_x.keys():
    print(key)

In [ ]:
%f \ [ ] 這是 feature columns, 我晚點可以嘗試把它弄成光一個 shape 為 (4,) 的單一 feature 
my_feature_columns = []
for key in train_x.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
my_feature_columns    

In [ ]:
%f \ 從 main 裡拆分出來，方便 tracing. 可看出若無指定 model directory 自動放在
%f \     AppData\Local\Temp\tmpijrg8mkm <-- 每次都不一樣，這樣好嗎？
%f \ 以及本 classifier 的種種細節
%f \ 

# Build 2 hidden layer DNN with 10, 10 units respectively.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 10 nodes each.
    hidden_units=[10, 10],
    # The model must choose between 3 classes.
    n_classes=3)

In [ ]:
%f \ 複習一下 lambda --> 
%f \ lambda 是個 function, 產生 key(i) 取 x 的第 i 個 element。用於 sorted() 時指定哪個當 key 
s = [('a', 3), ('b', 2), ('c', 1)]
key = lambda x:x[1]
print(sorted(s, key=key))
%f \ 單獨使用時，看起來不太一樣，其實做的是一樣的事情。
%f key :> ([('a',3),('b',2),('c',1),('d',4)]) tib.
%f key tib.
%f key .source
%f \ 當沒有給 lambda argument 時，是怎樣？
x = lambda:s
%f x tib. \ 直接傳回 : 之後的東西(執行後)的傳回值
%f x type tib.
%f \ 查看 x 的 source code 
%f x .source
%f x :> () tib.

%f \ 所以，input_fn = lambda:iris_data.train_input_fn(train_x, train_y,args.batch_size)
%f \ 當中的 iris_data.train_input_fn() 是傳回一個 function 但是該 function 的 argument 是活的，
%f \ 要延後到最後一秒鐘才確定，因此要用 lambda 或單純的 function 包一層用來指定 arguments。
    

In [ ]:
%f \ 這就開始訓練了 . . .  
%f \ 

# Train the Model.
classifier.train(
    input_fn=lambda:iris_data.train_input_fn(train_x, train_y,
                                             args.batch_size),
    steps=args.train_steps)

這時候的 classifier 已經受過訓練了! 可以接受考核了

In [ ]:
# Evaluate the model.
eval_result = classifier.evaluate(
    input_fn=lambda:iris_data.eval_input_fn(test_x, test_y,
                                            args.batch_size))

In [ ]:
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

In [ ]:
%f \ 複習一下 **eval_result 啥意思？
%f eval_result . cr
%f eval_result :> ['accuracy'] . cr
{**eval_result}

In [ ]:
rslt = {'accuracy': 0.96666664, 'average_loss': 0.056607112, 'loss': 1.6982133, 'global_step': 1000}
print('Test set accuracy: {loss:0.3f}'.format(**rslt))
print('Test set accuracy: {:0.3f}'.format(r['accuracy']))
print('Test set accuracy: {a:0.3f}'.format(**{'a': 11,'b': 22}))
print('Test set accuracy: {b:0.3f}'.format(**{'a': 33,'b': 44}))
%f \ 最後一式拆解不成功，拆開來的寫法也沒意義，要嘛直接寫就好了。
%f \ print('Test set accuracy: {b:0.3f}'.format(({'a': 55},{'b': 66})))

In [ ]:
# 這一段的意思是, 準備了 3 或 4 組資料 predict_x 分別的正確答案如 expected 所列
# Generate predictions from the model
expected = ['Setosa', 'Versicolor', 'Virginica','Versicolor']
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9,6.2],
    'SepalWidth': [3.3, 3.0, 3.1,2.9],
    'PetalLength': [1.7, 4.2, 5.4,4.3],
    'PetalWidth': [0.5, 1.5, 2.1,1.3],
}

[(i,j) for i,j in zip(predict_x,expected)]
[i for i in zip(predict_x,expected)]


In [ ]:
predictions = classifier.predict(
    input_fn=lambda:iris_data.eval_input_fn(predict_x,
                                            labels=None,
                                            batch_size=args.batch_size))

In [ ]:
# 這樣寫也可以
predictions = classifier.predict(
    lambda:iris_data.eval_input_fn(predict_x,
                                            labels=None,
                                            batch_size=args.batch_size))

In [ ]:
%f predictions dir . cr

In [ ]:
%f \ 以上 predictions 瞬間就取得了
%f predictions tib.
%f \ 這 'generator' 用過就沒了，要再看一遍就得重新跑一次 classifier.predict()
pred = [i for i in predictions]
pred

In [ ]:
template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')

In [ ]:
for pred_dict, expec in zip(predictions, expected):
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print(template.format(iris_data.SPECIES[class_id],
                          100 * probability, expec))

In [ ]:
zip?
%f \ zip() 傳回 iterator 像拉拉鍊一樣，所有第一個一組、所有第二個一組、如此以往到哪個先結束為止
z = zip((1,2,3,4),(11,22,33),{'aa':1,'bb':2})
%f z type tib.
z1 = [i for i in z]
z2 = [i for i in z]
%f z1 tib.
%f z2 tib.

In [ ]:
[(pred_dict, expec) for pred_dict, expec in zip(pred, expected)]

所以賣弄這個方法時 pred （測試資料）的個數與 expected （標準結果）就要吻合才有意思。以下簡單就是 prediction 與標準答案的比對列出來看。

In [ ]:
for pred_dict, expec in zip(pred, expected):
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print(template.format(iris_data.SPECIES[class_id],
                          100 * probability, expec))

# 以下是原本的教學程式

### premade_estimator.py
c:\Users\hcche\Documents\GitHub\models\samples\core\get_started\premade_estimator.py 

In [ ]:
def main(argv):
    args = parser.parse_args("")  # default is (["--batch_size=100", "--train_steps=1000"])
    # Fetch the data
    (train_x, train_y), (test_x, test_y) = iris_data.load_data()

    # Feature columns describe how to use the input.
    my_feature_columns = []
    for key in train_x.keys():
        my_feature_columns.append(tf.feature_column.numeric_column(key=key))

    # Build 2 hidden layer DNN with 10, 10 units respectively.
    classifier = tf.estimator.DNNClassifier(
        feature_columns=my_feature_columns,
        # Two hidden layers of 10 nodes each.
        hidden_units=[10, 10],
        # The model must choose between 3 classes.
        n_classes=3)

    # Train the Model.
    classifier.train(
        input_fn=lambda:iris_data.train_input_fn(train_x, train_y,
                                                 args.batch_size),
        steps=args.train_steps)

    # Evaluate the model.
    eval_result = classifier.evaluate(
        input_fn=lambda:iris_data.eval_input_fn(test_x, test_y,
                                                args.batch_size))

    print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

    # Generate predictions from the model
    expected = ['Setosa', 'Versicolor', 'Virginica']
    predict_x = {
        'SepalLength': [5.1, 5.9, 6.9],
        'SepalWidth': [3.3, 3.0, 3.1],
        'PetalLength': [1.7, 4.2, 5.4],
        'PetalWidth': [0.5, 1.5, 2.1],
    }

    predictions = classifier.predict(
        input_fn=lambda:iris_data.eval_input_fn(predict_x,
                                                labels=None,
                                                batch_size=args.batch_size))

    template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')

    for pred_dict, expec in zip(predictions, expected):
        class_id = pred_dict['class_ids'][0]
        probability = pred_dict['probabilities'][class_id]

        print(template.format(iris_data.SPECIES[class_id],
                              100 * probability, expec))


if __name__ == '__main__':
    tf.logging.set_verbosity(tf.logging.INFO)
    tf.app.run(main)

### iris_data.py
c:\Users\hcche\Documents\GitHub\models\samples\core\get_started\iris_data.py

In [ ]:
import pandas as pd
import tensorflow as tf

TRAIN_URL = "http://download.tensorflow.org/data/iris_training.csv"
TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth',
                    'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

def maybe_download():
    train_path = tf.keras.utils.get_file(TRAIN_URL.split('/')[-1], TRAIN_URL)
    test_path = tf.keras.utils.get_file(TEST_URL.split('/')[-1], TEST_URL)

    return train_path, test_path

# 這個有用到
def load_data(y_name='Species'):
    """Returns the iris dataset as (train_x, train_y), (test_x, test_y)."""
    train_path, test_path = maybe_download()

    train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
    train_x, train_y = train, train.pop(y_name)

    test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
    test_x, test_y = test, test.pop(y_name)

    return (train_x, train_y), (test_x, test_y)

# 這個有用到
def train_input_fn(features, labels, batch_size):
    """An input function for training"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)

    # Return the dataset.
    return dataset


# 這個有用到
def eval_input_fn(features, labels, batch_size):
    """An input function for evaluation or prediction"""
    features=dict(features)
    if labels is None:
        # No labels, use only features.
        inputs = features
    else:
        inputs = (features, labels)

    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices(inputs)

    # Batch the examples
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)

    # Return the dataset.
    return dataset


# The remainder of this file contains a simple example of a csv parser,
#     implemented using the `Dataset` class.

# `tf.parse_csv` sets the types of the outputs to match the examples given in
#     the `record_defaults` argument.
CSV_TYPES = [[0.0], [0.0], [0.0], [0.0], [0]]

def _parse_line(line):
    # Decode the line into its fields
    fields = tf.decode_csv(line, record_defaults=CSV_TYPES)

    # Pack the result into a dictionary
    features = dict(zip(CSV_COLUMN_NAMES, fields))

    # Separate the label from the features
    label = features.pop('Species')

    return features, label


def csv_input_fn(csv_path, batch_size):
    # Create a dataset containing the text lines.
    dataset = tf.data.TextLineDataset(csv_path).skip(1)

    # Parse each line.
    dataset = dataset.map(_parse_line)

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)

    # Return the dataset.
    return dataset


    ----- play ground -----

In [ ]:
# This cell creates %%csv cell magic. See http://ipython-books.github.io/14-creating-an-ipython-extension-with-custom-magic-commands
import pandas as pd
from io import StringIO
from IPython.core.magic import (register_cell_magic)
@register_cell_magic
def csv(line, cell):
    return pd.read_csv(StringIO(cell), sep=line, engine='python')

In [ ]:
%%csv \s+
a b c
1 2 3
4 5 6
7 8 9

In [ ]:
%f words module
%f help modul

In [ ]:
%f none value locals
def smallworld():
    x = 11
    y = 22
    peforth.push(locals()).dictate('to locals').ok

In [ ]:
peforth.ok()    

In [ ]:
input_fn=lambda:iris_data.train_input_fn(train_x, train_y, args.batch_size)
%f \ lambda:iris_data.train_input_fn(_x, _y, _size)  # 定義時 identifiers 可以亂寫,那這好像是 macro 了?
%f input_fn constant i/f
%f i/f . cr
%f i/f type . cr
%f i/f dir . cr
%f i/f :> () . cr

In [ ]:
f = lambda:locals

def outer():
    x = '1'
    y = '2'
    
    def given(locs):
        return(locs)
    
    def getit(locs=locals()):
        return(locs) 
    
    def getit2(locs=locals):
        return(locs()) 
    
    def testgiven():
        x = 33
        y = 44
        l = given(locals())
        return(x,y,l)
    
    def testgetit():
        x = 55
        y = 66
        l = getit()
        return(x,y,l)
    
    def testgetit2():
        x = 77
        y = 88
        l = getit2()
        return(x,y,l)

    return testgiven(), testgetit(), testgetit2()

outer()

In [ ]:
f = input_fn()

In [ ]:
f

In [ ]:
def myDebugger(loc):
    print(loc)

def debuggee():
    x = 11
    peforth.ok('11>', loc=dict(locals()))
    x = 22
    peforth.ok('22>', loc=dict(locals()))
    x = 33
    peforth.ok('33>', loc=dict(locals()))

debuggee()    

